In [2]:
from scipy.io import loadmat
from PIL import Image, ImageDraw
from IPython.display import Image as IPImage
# import os

import numpy as np
import pandas as pd
import os

# from PIL import Image

import torch

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional  as TF

# from utils import img_utils
import matplotlib.pyplot as plt

In [3]:
CUR_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")))
CUR_DIR

'/Users/tonmoy/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Education Project/Gaze/src/gazepoint/gfie/test'

In [4]:
WORKSPACE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(CUR_DIR)))))
DATASET_DIR = os.path.join(WORKSPACE_DIR, "Data", "gaze360")
DATASET_DIR

'/Users/tonmoy/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Education Project/Data/gaze360'

In [5]:
mat = loadmat(os.path.join(CUR_DIR, "metadata.mat"))
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'recordings', 'recording', 'frame', 'ts', 'target_cam', 'target_pos3d', 'target_pos2d', 'person_identity', 'person_cam', 'person_eyes3d', 'person_eyes2d', 'person_body_bbox', 'person_head_bbox', 'person_face_bbox', 'person_eye_left_bbox', 'person_eye_right_bbox', 'gaze_dir', 'splits', 'split'])

In [6]:
mat["recordings"]

array([[array(['rec_000'], dtype='<U7'), array(['rec_001'], dtype='<U7'),
        array(['rec_002'], dtype='<U7'), array(['rec_003'], dtype='<U7'),
        array(['rec_004'], dtype='<U7'), array(['rec_005'], dtype='<U7'),
        array(['rec_006'], dtype='<U7'), array(['rec_007'], dtype='<U7'),
        array(['rec_008'], dtype='<U7'), array(['rec_009'], dtype='<U7'),
        array(['rec_010'], dtype='<U7'), array(['rec_011'], dtype='<U7'),
        array(['rec_012'], dtype='<U7'), array(['rec_013'], dtype='<U7'),
        array(['rec_014'], dtype='<U7'), array(['rec_015'], dtype='<U7'),
        array(['rec_016'], dtype='<U7'), array(['rec_017'], dtype='<U7'),
        array(['rec_018'], dtype='<U7'), array(['rec_019'], dtype='<U7'),
        array(['rec_020'], dtype='<U7'), array(['rec_021'], dtype='<U7'),
        array(['rec_022'], dtype='<U7'), array(['rec_023'], dtype='<U7'),
        array(['rec_024'], dtype='<U7'), array(['rec_025'], dtype='<U7'),
        array(['rec_026'], dtype='<U7'

In [7]:
mat["recording"][0]

array([ 0,  0,  0, ..., 79, 79, 79], dtype=int32)

In [8]:
mat["frame"]

array([[  0,   1,   2, ..., 521, 522, 522]])

In [9]:
mat["ts"]

array([[ 0.        ,  0.12525296,  0.25050497, ..., 63.37850809,
        63.50351596, 63.50351596]])

In [10]:
mat["person_identity"][0][0]

15

In [11]:
mat["target_pos3d"]

array([[ 1.69389339,  1.75942589, -0.62393989],
       [ 1.68793889,  1.76528217, -0.62475749],
       [ 1.68868502,  1.76575734, -0.62507184],
       ...,
       [ 0.67011673, -0.48928031, -0.51240202],
       [ 0.64817876, -0.47156056, -0.44699191],
       [ 0.64817876, -0.47156056, -0.44699191]])

In [12]:
mat["person_head_bbox"]

array([[0.58718794, 0.4859066 , 0.04633445, 0.0382576 ],
       [0.5837803 , 0.48372877, 0.04876879, 0.04026759],
       [0.5783111 , 0.47928429, 0.05532767, 0.04568315],
       ...,
       [0.51265723, 0.47321415, 0.08451395, 0.06978178],
       [0.49715102, 0.47933304, 0.08145172, 0.06725335],
       [0.51202176, 0.47195637, 0.08704371, 0.07187057]])

In [59]:
def prepare_dataset(mat, dstype):
    orig_x, orig_y = 3382, 4096
    frame = mat["frame"]
    # person_eyes_2d = mat["person_eyes2d"]
    # person_eyes_3d = mat["person_eyes3d"]

    himg = list()
    simg = list()
    frame_id = list()
    hbbox = list()
    fbbox = list()
    bbbox = list()
    person_eyes2d = list()
    person_eyes3d = list()
    gaze_direction = list()
    target2d = list()
    target3d = list()


    for i in range(len(frame[0])):
        if  mat["split"][0][i] == dstype:
            himg += [os.path.join(
                mat["recordings"][0][mat["recording"][0][i]].item(),
                "head",
                "%06d" % mat["person_identity"][0][i].item(),
                "%06d.jpg" % mat["frame"][0][i].item(),
            )]
            simg += [os.path.join(
                mat["recordings"][0][mat["recording"][0][i]].item(),
                "body",
                "%06d" % mat["person_identity"][0][i].item(),
                "%06d.jpg" % mat["frame"][0][i].item(),
            )]
            frame_id += [mat["frame"][0][i]]
            hbbox += [mat["person_head_bbox"][i]]
            # fbbox += [mat["person_face_bbox"][i]]
            # bbbox += [mat["person_body_bbox"][i]]
            person_eyes2d += [mat["person_eyes2d"][i]]
            person_eyes3d += [mat["person_eyes3d"][i]]
            gaze_direction += [mat["gaze_dir"][i]]
            target2d += [mat["target_pos2d"][i]]
            target3d += [mat["target_pos3d"][i]]


    df = pd.DataFrame({
        "simg": simg,
        "himg": himg,
        "frame_id": frame_id,
        "h_x_min" : [x[0] for x in hbbox],
        "h_y_min" : [x[1] for x in hbbox],
        "h_x_max" : [x[2] for x in hbbox],
        "h_y_max" : [x[3] for x in hbbox],
        "eye_u" : [x[0] for x in person_eyes2d],
        "eye_v" : [x[1] for x in person_eyes2d],
        "eye_X" : [x[0] for x in person_eyes3d],
        "eye_Y" : [x[1] for x in person_eyes3d],
        "eye_Z" : [x[2] for x in person_eyes3d],
        "gaze_dirX": [x[0] for x in gaze_direction],
        "gaze_dirY": [x[1] for x in gaze_direction],
        "gaze_dirZ": [x[2] for x in gaze_direction],
        "gaze_u" : [x[0] for x in target2d],
        "gaze_v": [x[1] for x in target2d],
        "gaze_X": [x[0] for x in target3d],
        "gaze_Y": [x[1] for x in target3d],
        "gaze_Z": [x[2] for x in target3d],
    })

    return df



In [71]:
## Prepare Train Dataset
df = prepare_dataset(mat, 0)
df.to_csv(os.path.join(DATASET_DIR, "train.csv"))
df.shape

(126928, 20)

In [72]:
## Prepare Validation Dataset
df = prepare_dataset(mat, 1)
df.to_csv(os.path.join(DATASET_DIR, "validation.csv"))
df.shape

(17038, 20)

In [73]:
## Prepare Test Dataset
df = prepare_dataset(mat, 2)
df.to_csv(os.path.join(DATASET_DIR, "test.csv"))
df.shape

(25969, 20)

In [13]:
# img_path = os.path.join(DATASET_DIR, "imgs", mat["recordings"]) # , "body", '%06d' % mat["person_identity"][0], "%06d.jpg" % mat["frame"][0]
# img_path
orig_x, orig_y = 3382, 4096
recordings = mat["recordings"]
recording = mat["recording"]
splits = mat["splits"]
split = mat["split"]
person_head_bbox = mat["person_head_bbox"]
person_face_bbox = mat["person_face_bbox"]
person_body_bbox = mat["person_body_bbox"]
person_identity = mat["person_identity"]
gaze_dir = mat["gaze_dir"]
frame = mat["frame"]
person_eyes_2d = mat["person_eyes_2d"]
person_eyes_3d = mat["person_eyes_3d"]

i = 7370
img_path = os.path.join(
    DATASET_DIR,
    "imgs",
    recordings[0][recording[0][i]].item(),
    "head",
    "%06d" % mat["person_identity"][0][i].item(),
    "%06d.jpg" % mat["frame"][0][i].item(),
)
# img_path
# img = Image.open(img_path)
# Image._show(img)

In [80]:
from transformers import pipeline

checkpoint = "vinvino02/glpn-nyu"
depth_estimator = pipeline("depth-estimation", model=checkpoint, device=0)

sample_img = Image.open(os.path.join(DATASET_DIR, "imgs", lines[0].split(" ")[0]))
preds = depth_estimator(sample_img)

In [82]:
sample_img.size

(217, 217)

In [85]:
preds

{'predicted_depth': tensor([[[5.3033, 5.9054, 5.7112,  ..., 4.9678, 5.0379, 5.1203],
          [6.2931, 7.2684, 7.4032,  ..., 4.8657, 5.0426, 4.9449],
          [6.5597, 7.6745, 7.8852,  ..., 5.2611, 5.4131, 5.0675],
          ...,
          [3.2218, 3.0695, 2.8769,  ..., 5.6232, 5.4847, 5.1200],
          [3.4083, 3.1831, 2.9930,  ..., 5.5539, 5.3422, 5.0289],
          [3.8051, 3.6194, 3.3994,  ..., 5.2701, 5.1984, 4.9153]]]),
 'depth': <PIL.Image.Image image mode=L size=217x217>}

In [86]:
preds["predicted_depth"].shape, preds["depth"].size

(torch.Size([1, 192, 192]), (217, 217))

In [22]:
class Gaze360(Dataset):

    def __init__(self,dstype,opt,show=False):
        self.dataset_root = opt.DATASET.root_dir
        # rgb_path=os.path.join(opt.DATASET.root_dir,opt.DATASET.rgb)
        # depth_path=os.path.join(opt.DATASET.root_dir,opt.DATASET.depth)

        # camerapara=np.load(os.path.join(opt.DATASET.root_dir,opt.DATASET.camerapara))

        if dstype=="train":
            annofile_path=os.path.join(opt.DATASET.root_dir,opt.DATASET.train)
        elif dstype=="valid":
            annofile_path = os.path.join(opt.DATASET.root_dir, opt.DATASET.valid)
        elif dstype=="test":
            annofile_path = os.path.join(opt.DATASET.root_dir, opt.DATASET.test)
        else:
            raise NotImplemented

        df=pd.read_csv(annofile_path)

        self.X_train = df[['simg', 'himg', "h_x_min","h_y_min","h_x_max","h_y_max",'eye_u','eye_v','eye_X','eye_Y','eye_Z']]
        self.Y_train = df[['gaze_u', 'gaze_v', 'gaze_X', 'gaze_Y', 'gaze_Z']]

        self.length=len(df)

        # self.rgb_path=rgb_path
        # self.depth_path=depth_path
        # self.camerapara=camerapara

        self.input_size=opt.TRAIN.input_size
        self.output_size=opt.TRAIN.output_size

        transform_list = []
        transform_list.append(transforms.Resize((self.input_size, self.input_size)))
        transform_list.append(transforms.ToTensor())
        transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

        self.transform = transforms.Compose(transform_list)

        self.dstype=dstype

        self.imshow=show

        self.depth_estimator = pipeline("depth-estimation", model="vinvino02/glpn-nyu", device=0)

    def __getitem__(self, index):

        # scene_id,frame_index,h_x_min,h_y_min,h_x_max,h_y_max,eye_u,eye_v,eye_X,eye_Y,eye_Z=self.X_train.iloc[index]
        simg, himg, h_x_min, h_y_min, h_x_max, h_y_max, eye_u, eye_v, eye_X, eye_Y, eye_Z = self.X_train.iloc[index]
        # scene_id=str(int(scene_id))
        # frame_index=int(frame_index)

        gaze_u, gaze_v,gaze_X, gaze_Y, gaze_Z = self.Y_train.iloc[index]

        # rgb_path=os.path.join(self.rgb_path,self.dstype,"scene{}".format(scene_id),"{:04}.jpg".format(frame_index))
        # depth_path=os.path.join(self.depth_path,self.dstype,"scene{}".format(scene_id),"{:04}.npy".format(frame_index))

        # load the rgb image
        img = Image.open(os.path.join(self.dataset_root, "imgs", simg))
        img = img.convert('RGB')
        width, height = img.size
        org_width, org_height = width, height

        # load the depth image
        # depthimg=np.load(depth_path)
        # # replace the invalid value with 0
        # depthimg[np.isnan(depthimg)]=0
        # depthimg=depthimg.astype(np.float32)
        # depthimg=Image.fromarray(depthimg)
        depthimg = depth_estimator(Image.open(os.path.join(self.dataset_root, "imgs", simg)))["depth"]
        depth_estimator = pipeline("depth-estimation", model="vinvino02/glpn-nyu", device=0)
        estimated_depth = depth_estimator(img)
        depthimg = estimated_depth["depth"]
        



        # expand face bbox a bit
        k=0.1
        h_x_min -= k * abs(h_x_max - h_x_min)
        h_y_min -= k * abs(h_y_max - h_y_min)
        h_x_max += k * abs(h_x_max - h_x_min)
        h_y_max += k * abs(h_y_max - h_y_min)

        x_min, y_min, x_max, y_max=map(float,[h_x_min,h_y_min,h_x_max,h_y_max])

        # Data augmentation for training procedure
        offset_x, offset_y = 0, 0
        flip_flag = False
        if self.dstype=="train":

            # Jitter (expansion-only) bounding box size
            if np.random.random_sample() <= 0.5:
                k = np.random.random_sample() * 0.2
                x_min -= k * abs(x_max - x_min)
                y_min -= k * abs(y_max - y_min)
                x_max += k * abs(x_max - x_min)
                y_max += k * abs(y_max - y_min)

            # Random crop
            if np.random.random_sample() <= 0.5:
                # calculate the minimum valid range of the crop that doesn't exclude the face and the gaze target
                crop_x_min = np.min([gaze_u , x_min, x_max])
                crop_y_min = np.min([gaze_v , y_min, y_max])
                crop_x_max = np.max([gaze_u , x_min, x_max])
                crop_y_max = np.max([gaze_v , y_min, y_max])

                # randomly select a top left corner
                if crop_x_min >= 0:
                    crop_x_min = np.random.uniform(0, crop_x_min)
                if crop_y_min >= 0:
                    crop_y_min = np.random.uniform(0, crop_y_min)

                # find the range of valid crop width and height starting from the (crop_x_min, crop_y_min)
                crop_width_min = crop_x_max - crop_x_min
                crop_height_min = crop_y_max - crop_y_min
                crop_width_max = width - crop_x_min
                crop_height_max = height - crop_y_min
                # randomly select a width and a height
                crop_width = np.random.uniform(crop_width_min, crop_width_max)
                crop_height = np.random.uniform(crop_height_min, crop_height_max)

                # crop scene img
                img = TF.crop(img, crop_y_min, crop_x_min, crop_height, crop_width)

                # crop depth img
                depthimg=TF.crop(depthimg, crop_y_min, crop_x_min, crop_height, crop_width)

                # record the crop's (x, y) offset
                offset_x, offset_y = crop_x_min, crop_y_min

                # convert coordinates into the cropped frame
                x_min, y_min, x_max, y_max = x_min - offset_x, y_min - offset_y, x_max - offset_x, y_max - offset_y

                gaze_u, gaze_v = (gaze_u - offset_x) / float(crop_width), \
                                 (gaze_v - offset_y) / float(crop_height)

                eye_u, eye_v = (eye_u - offset_x) / float(crop_width), \
                                 (eye_v  - offset_y) / float(crop_height)

                width, height = crop_width, crop_height

            else:
                gaze_u, gaze_v = (gaze_u - offset_x) / float(width), \
                                 (gaze_v - offset_y) / float(height)

                eye_u, eye_v = (eye_u - offset_x) / float(width), \
                                 (eye_v  - offset_y) / float(height)

            # Random flip
            if np.random.random_sample() <= 0.5:
                flip_flag=True
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
                depthimg = depthimg.transpose(Image.FLIP_LEFT_RIGHT)
                x_max_2 = width - x_min
                x_min_2 = width - x_max
                x_max = x_max_2
                x_min = x_min_2
                gaze_u= 1 - gaze_u
                eye_u=1 -eye_u

            # Random change the brightness, contrast and saturation of the scene images
            if np.random.random_sample() <= 0.5:
                img = TF.adjust_brightness(img, brightness_factor=np.random.uniform(0.5, 1.5))
                img = TF.adjust_contrast(img, contrast_factor=np.random.uniform(0.5, 1.5))
                img = TF.adjust_saturation(img, saturation_factor=np.random.uniform(0, 1.5))

        else:

            gaze_u, gaze_v = gaze_u  / float(width), \
                             gaze_v  / float(height)

            eye_u, eye_v = eye_u  / float(width), \
                           eye_v  / float(height)

        # represent the head location with mask
        head_channel = img_utils.get_head_box_channel(x_min, y_min, x_max, y_max, width, height,
                                                    resolution=self.input_size, coordconv=False).unsqueeze(0)

        # the final image size in train/valid/test
        final_width,final_height=img.size

        # crop the face
        headimg = img.crop((int(x_min), int(y_min), int(x_max), int(y_max)))

        # set for display
        if self.imshow:
            img_show=img
            depthimg_show=depthimg

        # resize scene/face image and convert them to tensor
        if self.transform is not None:

            img=self.transform(img)
            headimg=self.transform(headimg)

        # Generate the matrix_T
        depthmap=depthimg.resize((self.input_size,self.input_size),Image.BICUBIC)
        depthmap=np.array(depthmap)

        # scale proportionally
        scale_width,scale_height=final_width/self.input_size,final_height/self.input_size

        # construct empty matrix
        matrix_T_DW = np.linspace(0, self.input_size - 1, self.input_size)
        matrix_T_DH = np.linspace(0, self.input_size - 1, self.input_size)
        [matrix_T_xx, matrix_T_yy] = np.meshgrid(matrix_T_DW, matrix_T_DH)

        # construct matrix_T according to Eq 3. in paper
        fx,fy,cx,cy=self.camerapara
        if flip_flag:
            cx= org_width - cx
            matrix_T_X = (matrix_T_xx * scale_width + (org_width - final_width - offset_x) - cx) * depthmap / fx

        else:
            matrix_T_X = (matrix_T_xx * scale_width + offset_x - cx) * depthmap / fx

        matrix_T_Y = (matrix_T_yy * scale_height + offset_y - cy) * depthmap / fy
        matrix_T_Z = depthmap

        matrix_T = np.dstack((matrix_T_X, matrix_T_Y, matrix_T_Z))
        matrix_T = matrix_T.reshape([-1, 3])
        matrix_T = matrix_T.reshape([self.input_size, self.input_size, 3])

        if flip_flag:
            matrix_T = matrix_T - np.array([-eye_X, eye_Y, eye_Z])
        else:
            matrix_T = matrix_T - np.array([eye_X, eye_Y, eye_Z])

        norm_value = np.linalg.norm(matrix_T, axis=2, keepdims=True)
        norm_value[norm_value <= 0] = 1

        matrix_T = matrix_T / norm_value

        # convert it to tensor
        matrix_T=torch.from_numpy(matrix_T).float()


        # generate the gaze vector label
        gaze_vector = np.array([gaze_X - eye_X, gaze_Y - eye_Y, gaze_Z - eye_Z])

        if flip_flag:
            gaze_vector[0]=-gaze_vector[0]

        norm_gaze_vector = 1.0 if np.linalg.norm (gaze_vector) <= 0.0 else np.linalg.norm (gaze_vector)
        gaze_vector=gaze_vector/norm_gaze_vector
        gaze_vector=torch.from_numpy(gaze_vector)

        # generate the heat map label
        gaze_heatmap = torch.zeros(self.output_size, self.output_size)  # set the size of the output

        gaze_heatmap = img_utils.draw_labelmap(gaze_heatmap, [gaze_u * self.output_size, gaze_v * self.output_size],
                                               3,type='Gaussian')


        # auxilary info
        gaze_target2d=torch.from_numpy(np.array([gaze_u,gaze_v]))
        matrix_T_heatmap = np.dot(matrix_T, gaze_vector)

        # display
        if self.imshow:

            def unnorm(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
                std = np.array(std).reshape(3, 1, 1)
                mean = np.array(mean).reshape(3, 1, 1)
                return img * std + mean

            figure,ax=plt.subplots(2,3)
            figure.set_size_inches(15 ,8)

            simgshow=unnorm(img.numpy()) * 255
            simgshow=np.clip(simgshow,0,255)
            simgshow=simgshow.astype(np.uint8)

            himgshow=unnorm(headimg.numpy()) * 255
            himgshow=np.clip(himgshow,0,255)
            himgshow=himgshow.astype(np.uint8)

            eyes_outpix=[eye_u*self.input_size,eye_v*self.input_size]

            gaze_outpix = [gaze_u * self.input_size, gaze_v * self.input_size]

            # display scene image
            ax[0][0].imshow(np.transpose(simgshow, (1, 2, 0)))
            # display gaze target and eyes in scene image
            ax[0][0].scatter(eyes_outpix[0],eyes_outpix[1])
            ax[0][0].scatter(gaze_outpix[0],gaze_outpix[1])

            # display depth map
            ax[0][1].imshow(depthmap,cmap='gray')
            # display head image
            ax[1][0].imshow(np.transpose(himgshow, (1, 2, 0)))
            # display expected stero FoV heatmap
            ax[1][1].imshow(matrix_T_heatmap, cmap='jet')


            plt.show()

        all_data={}
        all_data['simg'] = img
        all_data["himg"] = headimg
        all_data["headloc"] = head_channel
        all_data["matrixT"]=matrix_T

        # Y_label
        all_data["gaze_heatmap"] = gaze_heatmap
        all_data["gaze_vector"] = gaze_vector
        all_data["gaze_target2d"] = gaze_target2d

        return all_data


    def __len__(self):

        return self.length